In [1]:
import numpy as np
import numpy.linalg as la
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import sys
import math
import inspect
from PIL import Image

In [2]:
def arr_to_pic(a):
    return Image.fromarray(np.uint8(a))

def pic_to_arr(img):
    return np.array(pic[:, :, 0])

In [41]:
def bytes_from_file(filename, chunksize=8192):
    with open(filename, 'rb') as f:
        while True:
            chunk = f.read(chunksize)
            if chunk:
                for b in chunk:
                    yield b
            else:
                break

def read_train_labels(filename, need_buben):
    all_bytes = list(bytes_from_file(filename))
    def read(idx=[0]):
        idx[0] += 1
        return all_bytes[idx[0] - 1]
    def read_i32():
        return (int(read()) << 24) + (int(read()) << 16) + (int(read()) << 8) + int(read())
    buben = read_i32()
    assert "buben must be %d" % need_buben, buben == need_buben
    count = read_i32()
    labels = [read() for _ in range(count)]
    return np.array(labels)

def read_train_images(filename, need_buben):
    all_bytes = list(bytes_from_file(filename))
    def read(idx=[0]):
        idx[0] += 1
        return all_bytes[idx[0] - 1]
    def read_i32():
        return (int(read()) << 24) + (int(read()) << 16) + (int(read()) << 8) + int(read())
    buben = read_i32()
    assert "buben must be %d" % need_buben, buben == need_buben
    count = read_i32()
    rows = read_i32()
    columns = read_i32()
    print(count, rows, columns, count * rows * columns, file=sys.stderr)
    imgs = np.zeros((count, rows, columns))
    for x in range(count):
        for i in range(rows):
            for j in range(columns):
                imgs[x, i, j] = read() / 255
    return imgs
    
labels = read_train_labels('train-labels.idx1-ubyte', 2049)
images = read_train_images('train-images.idx3-ubyte', 2051)

60000 28 28 47040000


In [4]:
for i in range(10):
    arr_to_pic(images[i]).save('pics/test_pic%02d.png' % i)

In [54]:
class Node:
    def __init__(self, func, ws):
        self.inodes = []
        self.onodes = []
        self.value = None
        self.func = func
        self.ws = ws
    def get(self):
        if self.value == None:
#             print("Value is None", file=sys.stderr)
            assert len(self.inodes) == len(self.ws)
            gets = [n.get() for n in self.inodes]
            arg = sum(w * x for (w, x) in zip(self.ws, gets))
            self.value = self.func(arg)
        return self.value
    def clear(self):
        self.value = None
    def add_onode(self, onode):
        self.onodes.append(onode)
    def add_inode(self, inode):
        self.inodes.append(inode)
    def isbayes(self):
        return False

class InputNode:
    def __init__(self, i, j):
        self.i = i
        self.j = j
        self.value = None
    def get(self, img=None):
        if self.value == None:
            self.value = img[self.i, self.j]
        return self.value
    def clear(self):
        self.value = None
    def add_onode(self, onode):
        pass
    def add_inode(self, inode):
        pass
    def isbayes(self):
        return False

class BayesUnit:
    def get(*args):
        return 1
    def clear():
        pass
    def add_onode(onode):
        pass
    def add_inode(inode):
        pass
    def isbayes():
        return True

def nodepipe(inode, onode):
    inode.add_onode(onode)
    onode.add_inode(inode)
    
class Layer:
    def __init__(self, nodes):
        self.nodes = nodes
    def add_node(self, node):
        self.nodes.append(node)
    def calc(self, *args, **kwargs):
        f = lambda node: node.get(*args)
        pool = kwargs['pool'] if 'pool' in kwargs.keys() else None
        if pool != None:
            return list(pool.map(f, self.nodes))
        else:
            return [f(node) for node in self.nodes]
    def clear(self):
        for node in self.nodes:
            node.clear()
    def size(self):
        return len(self.nodes)

def layerpipe(ilayer, olayer):
    for inode in ilayer.nodes:
        for onode in olayer.nodes:
            nodepipe(inode, onode)

In [52]:
def identity(x):
    return x

def sigmoid(x):
#     print("!!! x = %f" % x, file=sys.stderr)
    return 1 / (1 + np.exp(-x))

def gaussian(mean, variance):
    std = np.sqrt(variance)
    return lambda: np.random.normal(mean, std)

In [55]:
def build_net(rows, columns, mid_size):
    size = rows * columns
    layers = []
    
    input_layer = Layer([])
    input_layer.add_node(BayesUnit)
    for i in range(rows):
        for j in range(columns):
            node = InputNode(i, j)
            input_layer.add_node(node)
    layers.append(input_layer)
            
    mid_layer = Layer([])
    mid_layer.add_node(BayesUnit)
    for i in range(mid_size):
        csize = 1 + layers[-1].size()
        node = Node(sigmoid, [0] * csize)
        mid_layer.add_node(node)
    layers.append(mid_layer)
    
    output_layer = Layer([])
    for i in range(10):
        csize = 1 + layers[-1].size()
        node = Node(sigmoid, [0] * csize)
        output_layer.add_node(node)
    layers.append(output_layer)
    
    for i in range(len(layers) - 1):
        layerpipe(layers[i], layers[i + 1])
        
    np.random.seed(1321)
    for layer in layers:
        for node in layer.nodes:
            if isinstance(node, Node):
                variance = 2 / (len(node.inodes) + len(node.onodes))
                gen = gaussian(0, variance)
                node.ws = [gen() for _ in node.inodes]
    return layers

In [56]:
import multiprocessing.dummy
thread_pool = multiprocessing.dummy.Pool(5)

In [57]:
def apply_net(layers, image):
    for layer in layers:
        layer.clear()
    layers[0].calc(image, pool=thread_pool)
    return np.array(layers[-1].calc(pool=thread_pool))

In [58]:
rows, columns = images[0].shape
layers = build_net(rows, columns, 1000)

In [59]:
def train_local(layers, img, ans, lrate=0.1):
    cur_res = apply_net(layers, img)
    sig1 = np.zeros(cur_res.shape)
    for i in range(cur_res.shape[0]):
        sig1[i] = cur_res[i] * (1 - cur_res[i])

    epsM = cur_res - ans
    epsH = np.zeros(len(layers[-2].nodes))
    for h, pnode in enumerate(layers[-2].nodes):
        for m, node in enumerate(layers[-1].nodes):
            if node.isbayes():
                continue
            epsH[h] += epsM[m] * node.ws[h] * sig1[m]

    for m, node in enumerate(layers[-1].nodes):
        if node.isbayes():
            continue
        for h, pnode, in enumerate(layers[-2].nodes):
            node.ws[h] -= lrate * epsM[m] * sig1[m] * pnode.get()
    
    for h, pnode in enumerate(layers[-2].nodes):
        if pnode.isbayes():
            continue
        sigma = pnode.get() * (1 - pnode.get())
        for j, jnode in enumerate(layers[-3].nodes):
            pnode.ws[j] -= lrate * epsH[h] * jnode.get() * sigma
    return layers
            
def train_net(layers, images, answers, bsize=128, iters=1, epochs=1, lrate=0.1):
    cnt = len(images)
    bs, anss = [], []
    for i in range(0, cnt, bsize):
        j = min(i + bsize, cnt)
        bs.append(images[i : j, :])
        anss.append(answers[i : j, :])
    for ep in range(epochs):
        for b, a in zip(bs, anss):
            for it in range(iters):
                for img, ans in zip(b, a):
                    train_local(layers, img, ans, lrate)
    return layers

In [60]:
answers = np.zeros((labels.shape[0], 10))
for i in range(labels.shape[0]):
    answers[i, labels[i]] = 1

In [68]:
cnt = 10
imgs, anss = images[:cnt], answers[:cnt]
layers = train_net(layers, imgs, anss, bsize=1, iters=2, epochs=3)
for i in range(cnt):
    result = np.argmax(apply_net(layers, images[i]))
    print(labels[i], ':', result)

!
!
!


5 : 4
0 : 4
4 : 4
1 : 4
9 : 4
2 : 4
1 : 4
3 : 4
1 : 4
4 : 4


In [15]:
for i in range(100):
    img = images[i]
    res = apply_net(layers, img)
    print(np.argmax(res), end=' ', file=sys.stderr)
print(file=sys.stderr)

1 0 4 8 1 1 9 1 9 1 4 1 1 0 1 1 0 1 0 1 0 1 8 8 0 0 2 2 0 0 1 0 1 1 0 0 4 0 0 1 1 1 9 1 1 8 0 0 3 2 1 1 0 1 8 8 1 8 8 1 0 1 8 1 4 1 2 1 0 0 1 1 1 0 0 0 0 8 1 1 0 0 0 1 1 2 2 8 0 2 0 1 8 0 1 0 1 8 0 1 
